In [1]:
try:
    import torch
except:
    !pip install torch
    import torch

import pandas as pd

from torch.utils.data import Dataset, DataLoader
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score, classification_report, roc_auc_score

In [2]:
df = pd.read_csv('LLCP2023_clean.csv', keep_default_na=False, dtype='category')

### Training hyperparameters

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 25

Setting device for training

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


### Encoding and splitting data into training and testing sets

In [5]:
label_encoders = {}
for column in df.columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

In [6]:
X = df.drop(columns=['_MENT14D'])
y = df['_MENT14D']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Dataset class for BRFSS data

In [7]:
class BRFSSDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float).to(device)
        self.y = torch.tensor(y.values, dtype=torch.long).to(device)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = BRFSSDataset(X_train, y_train)
test_dataset = BRFSSDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

## Definition of Neural Network model

### Model structure

In [9]:
class MentalHealthClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.Tanh(),
            nn.Linear(256, 128),
            nn.Tanh(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        return self.linear_relu_stack(x)

input_size = X_train.shape[1]  # Number of features
num_classes = len(y.unique())  # Number of target classes
model = MentalHealthClassifier(input_size, num_classes).to(device)
print(model)

MentalHealthClassifier(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=26, out_features=256, bias=True)
    (1): Tanh()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): Tanh()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=3, bias=True)
    (7): Softmax(dim=1)
  )
)


### Training and testing loops

In [366]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Forward pass
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## Training of Neural Network model

In [367]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.104353  [   64/63899]
loss: 0.958752  [ 6464/63899]
loss: 0.970720  [12864/63899]
loss: 0.836235  [19264/63899]
loss: 0.989137  [25664/63899]
loss: 0.870260  [32064/63899]
loss: 0.908456  [38464/63899]
loss: 0.936252  [44864/63899]
loss: 0.921477  [51264/63899]
loss: 0.851948  [57664/63899]
Test Error: 
 Accuracy: 67.6%, Avg loss: 0.864116 

Epoch 2
-------------------------------
loss: 0.916796  [   64/63899]
loss: 0.880595  [ 6464/63899]
loss: 0.816753  [12864/63899]
loss: 0.884217  [19264/63899]
loss: 0.966761  [25664/63899]
loss: 0.799839  [32064/63899]
loss: 0.838053  [38464/63899]
loss: 0.861884  [44864/63899]
loss: 0.901310  [51264/63899]
loss: 0.842948  [57664/63899]
Test Error: 
 Accuracy: 68.6%, Avg loss: 0.851460 

Epoch 3
-------------------------------
loss: 0.824647  [   64/63899]
loss: 0.910886  [ 6464/63899]
loss: 0.844478  [12864/63899]
loss: 0.834769  [19264/63899]
loss: 0.800798  [25664/63899]
loss: 0.809346  [32064/638

## Evaluation

In [368]:
y_pred_prob = []
y_pred = []
y_true = []
model.eval()
with torch.inference_mode():
    for X, y in test_loader:
        prob = model(X).to('cpu')
        y_pred_prob += prob.tolist()
        pred = prob.argmax(axis=1)
        y_pred += pred.tolist()
        y_true += y.to('cpu').tolist()

In [369]:
accuracy_score(y_true, y_pred)

0.6980281690140845

In [370]:
f1_score(y_true, y_pred, average='weighted')

0.6936582354872839

In [371]:
classification_report(y_true, y_pred, output_dict=True)

{'0': {'precision': 0.6752399232245682,
  'recall': 0.49985791418016484,
  'f1-score': 0.5744611365120836,
  'support': 3519.0},
 '1': {'precision': 0.6992884783422794,
  'recall': 0.7521482277121375,
  'f1-score': 0.7247558056795395,
  'support': 7448.0},
 '2': {'precision': 0.7072214965478634,
  'recall': 0.7567891373801917,
  'f1-score': 0.7311662004437156,
  'support': 5008.0},
 'accuracy': 0.6980281690140845,
 'macro avg': {'precision': 0.6939166327049037,
  'recall': 0.6695984264241647,
  'f1-score': 0.6767943808784462,
  'support': 15975.0},
 'weighted avg': {'precision': 0.6964779424871519,
  'recall': 0.6980281690140845,
  'f1-score': 0.6936582354872839,
  'support': 15975.0}}

In [372]:
roc_auc_score(y_true, y_pred_prob, multi_class='ovo')

0.8525919493674073

## Save trained model weights

In [374]:
torch.save(model.state_dict(), './nn_model.pt')

### Load model

In [10]:
model.load_state_dict(torch.load('./nn_model.pt', weights_only=True))
model.eval()

MentalHealthClassifier(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=26, out_features=256, bias=True)
    (1): Tanh()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): Tanh()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=3, bias=True)
    (7): Softmax(dim=1)
  )
)